In [1]:
import datetime
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import sys
import os

# Add the deployment folder to the system path so we can import utils
sys.path.append(os.path.abspath('../deployment'))

def clean_currency(x):
    """
    Converts '$8,400' string to 8400.0 float.
    NOW ROBUST: Returns None if it encounters text like 'Porsche'.
    """
    if pd.isna(x):
        return None
    if isinstance(x, str):
        # Remove symbols
        x_clean = x.replace('$', '').replace(',', '').strip()
        try:
            return float(x_clean)
        except ValueError:
            # If we can't convert it (e.g. it's text), return None
            return None
    return float(x)

def clean_mileage(x):
    """
    Converts '53,700' string to 53700.0 float.
    NOW ROBUST: Returns None if it encounters text.
    """
    if pd.isna(x):
        return None
    if isinstance(x, str):
        x_clean = x.replace(',', '').strip()
        try:
            return float(x_clean)
        except ValueError:
            return None
    return float(x)

def clean_model(x):
    if pd.isna(x):
        return x
    return x.replace('\nSave', '').strip()

def clean_and_group_title(x):
    if pd.isna(x):
        return 'Unknown' 
    
    # Apply original split and strip, then lowercase to handle casing issues
    val = str(x).split('(')[0].strip().lower()
    
    # 1. Clean Titles
    if val in ['clean', 'clen']:
        return 'Clean'
    
    # 2. Rebuilt / Salvage / Restored
    if any(keyword in val for keyword in ['rebuilt', 'salvage', 'reconstructed', 'totaled', 'restored']):
        return 'Rebuilt/Salvage'
        
    # 3. Mileage & Odometer Issues
    if any(keyword in val for keyword in ['mileage', 'odometer', 'mechanical limits']):
        return 'Mileage Issue'
        
    # 4. Lemon Law / Manufacturer Buyback
    if 'buyback' in val:
        return 'Buyback'
        
    # 5. Alternate Documentation (Bill of sale, Registered only, etc.)
    if val in ['bill of sale', 'no title', 'registered']:
        return 'Alternate Doc'
        
    # Catch-all for anything missed
    return 'Other'

def clean_transmission_type(x):
    if pd.isna(x):
        return "Unknown"
        
    # Split on parenthesis and strip trailing spaces
    val = str(x).split('(')[0].strip()
    
    # Strictly enforce valid types to filter out the charity auction paragraphs
    if val in ['Automatic', 'Manual']:
        return val
        
    return 'Other'

def extract_gears(x):
    if pd.isna(x):
        return None
        
    x_str = str(x)
    
    # 1. Improved Regex: 
    # (?i) makes it case-insensitive (catches "speed" and "Speed")
    # [-\s] allows either a hyphen or a space
    match = re.search(r'(?i)(\d+)[-\s]speed', x_str)
    
    if match:
        return float(match.group(1))
        
    # 2. Handle CVTs explicitly (usually coded as 1 gear for ML purposes)
    if 'CVT' in x_str.upper():
        return 1.0
        
    return None

def extract_engine_info(x):
    # 1. Handle entirely missing/blank values
    if pd.isna(x) or str(x).strip() == '':
        return None, "Unknown"  # None for numeric, "Unknown" for categorical
    
    # --- Fix Displacement ---
    disp_l = re.search(r'(\d+\.?\d*)\s*L', x, re.IGNORECASE)
    disp_cc = re.search(r'(\d+)\s*cc', x, re.IGNORECASE)
    disp_ci = re.search(r'(\d+)\s*ci', x, re.IGNORECASE)
    
    # Default to None for missing numeric values
    d_val = None 
    if disp_l:
        d_val = float(disp_l.group(1))
    elif disp_cc:
        d_val = round(float(disp_cc.group(1)) / 1000.0, 1) 
    elif disp_ci:
        d_val = round(float(disp_ci.group(1)) / 61.0237, 1) 
        
    # --- Fix Cylinders ---
    c_val = "Other" 
    
    cyl = re.search(r'([VIW])[- ]?(\d+)', x, re.IGNORECASE)
    flat = re.search(r'Flat[- ]?(\d+)', x, re.IGNORECASE)
    inline = re.search(r'Inline[- ]?(\d+)', x, re.IGNORECASE)
    l_typo = re.search(r'(l)(\d+)', x)
    
    if cyl:
        c_val = cyl.group(1).upper() + cyl.group(2)
    elif inline:
        c_val = 'I' + inline.group(1)
    elif flat:
        c_val = 'H' + flat.group(1) 
    elif l_typo:
        c_val = 'I' + l_typo.group(2)
    elif 'Rotary' in x:
        c_val = 'Rotary'
    elif 'Electric' in x or 'Motor' in x:
        c_val = 'Electric'
        
    return d_val, c_val

def get_main_color(x):
    # 1. Catch missing values and group them into "Other"
    if pd.isna(x):
        return "Other"
        
    # 2. Grab the primary color before slashes or " and "
    x = str(x).split('/')[0].split(' and ')[0].strip()
    x_lower = x.lower()
    
    # 3. Check for specific keywords FIRST to prevent substring collisions 
    special_map = {
        # Edge cases, Collisions & Exterior Bleed-over
        'titanium': 'Gray', 'titan': 'Black', 'mustang': 'Brown', 'tanzanite': 'Blue',
        'stainless': 'Silver', 'mercury': 'Silver', 'magnetic': 'Gray', 'thunder': 'Gray',
        
        # Blacks / Darks
        'ebony': 'Black', 'nero': 'Black', 'carbon': 'Black', 'onyx': 'Black', 
        'jet': 'Black', 'obsidian': 'Black', 'beluga': 'Black', 'panther': 'Black',
        'amido': 'Black', 'midnight': 'Black', 'anthracite': 'Gray', 'zebra': 'Black',
        
        # Grays / Silvers
        'granite': 'Gray', 'charcoal': 'Gray', 'graphite': 'Gray', 'slate': 'Gray', 
        'ash': 'Gray', 'agate': 'Gray', 'stone': 'Gray', 'shale': 'Gray', 
        'platinum': 'Gray', 'pewter': 'Gray', 'palladium': 'Gray', 'meteor': 'Gray',
        'flint': 'Gray', 'ocean': 'Gray',
        
        # Whites / Lights
        'chalk': 'White', 'ivory': 'White', 'pearl': 'White', 'porcelain': 'White', 
        'alabaster': 'White', 'bianco': 'White', 'magnolia': 'White', 'oyster': 'White',
        'ice': 'White', 'ceramic': 'White',
        
        # Beiges / Tans / Browns
        'parchment': 'Beige', 'linen': 'Beige', 'cream': 'Beige', 'ecru': 'Beige', 
        'luxor': 'Beige', 'cashmere': 'Beige', 'savanna': 'Beige', 'almond': 'Beige', 
        'bamboo': 'Beige', 'wheat': 'Beige', 'champagne': 'Beige', 'kalahari': 'Beige', 
        'gobi': 'Beige', 'macchiato': 'Beige', 'taupe': 'Beige', 'sand': 'Beige', 
        'dune': 'Beige', 'saddle': 'Brown', 'oak': 'Brown', 'cocoa': 'Brown', 
        'cognac': 'Brown', 'caramel': 'Brown', 'cuoio': 'Brown', 'cinnamon': 'Brown', 
        'java': 'Brown', 'havanna': 'Brown', 'havana': 'Brown', 'mocha': 'Brown', 
        'espresso': 'Brown', 'nougat': 'Brown', 'chestnut': 'Brown', 'amaro': 'Brown', 
        'sepia': 'Brown', 'truffle': 'Brown', 'walnut': 'Brown', 'tartufo': 'Brown', 
        'terra': 'Brown', 'natural': 'Brown', 'palomino': 'Tan', 'camel': 'Tan', 
        'khaki': 'Tan', 'atacama': 'Tan',
        
        # Reds / Oranges
        'salsa': 'Red', 'coral': 'Red', 'imola': 'Red', 'fox': 'Red', 
        'burgundy': 'Red', 'magma': 'Red', 'carrera': 'Red', 'maroon': 'Red', 
        'chateau': 'Red', 'bordeaux': 'Red', 'fiona': 'Red', 'scarlet': 'Red', 
        'garnet': 'Red', 'crimson': 'Red', 'ruby': 'Red', 'cabernet': 'Red', 
        'rosso': 'Red', 'sakhir': 'Orange', 'kyalami': 'Orange',
        
        # Greens / Blues
        'jade': 'Green', 'cypress': 'Green', 'forest': 'Green', 'nordkap': 'Blue', 
        'nautic': 'Blue', 'yachting': 'Blue', 'estoril': 'Blue', 'marina': 'Blue'
    }
    
    for key, val in special_map.items():
        if key in x_lower:
            return val

    # 4. Check standard baseline colors
    std_colors = [
        'black', 'white', 'gray', 'grey', 'silver', 'red', 'blue', 
        'green', 'brown', 'beige', 'yellow', 'orange', 'gold', 'purple', 'tan'
    ]
    
    for color in std_colors:
        if color in x_lower:
            return 'Gray' if color == 'grey' else color.capitalize()
            
    # 5. Everything else becomes "Other"
    return "Other"

def clean_seller_type(x):
    if pd.isna(x):
        return "Unknown"
        
    val = str(x)
    
    # Consolidate all Dealer types (ignores doc fees, etc.)
    if 'Dealer' in val:
        return 'Dealer'
        
    # Consolidate all Private Party types (ignores liens, temporary tags, \n, etc.)
    elif 'Private Party' in val:
        return 'Private Party'

    else:
        return 'Other'

def categorize_mods(text):
    """Categorizes modification severity."""
    if pd.isna(text): return "stock"
    text = str(text).lower()
    if "stock" in text and len(text) < 20: return "stock"
    
    if re.search(r'\b(turbo kit|supercharger|engine swap|ls swap|k20|roll cage|drilled|turbo|suspension)\b', text):
        return "heavy_mod"
    if re.search(r'\b(exhaust|muffler|intake|wheels|coilovers|springs|tint|bluetooth|screen|touchscreen|hitch|emblem)\b', text):
        return "light_mod"
    
    return "unknown_mod"

def calculate_flaw_severity(row):
    """Calculates a numeric penalty score for flaws."""
    # Handle missing columns gracefully
    flaws = str(row.get('Known Flaws', ''))
    notes = str(row.get('Seller Notes', ''))
    text = (flaws + " " + notes).lower()
    
    score = 0
    # Tier 1: Catastrophic
    if re.search(r'\b(salvage|rebuilt|branded|flood|frame damage|rolled)\b', text): score += 50
    if re.search(r'\b(tmu|true mileage unknown|odometer broken)\b', text): score += 40
    if re.search(r'\b(rust holes|rot|perforated|frame rust)\b', text): score += 40
    if re.search(r'\b(knock|smoke|overheat|head gasket|trans slip)\b', text): score += 30

    # Tier 2: Expensive
    if re.search(r'\b(leak|seep|drip|fluid)\b', text): score += 10
    if re.search(r'\b(crack|tear|rip|worn)\b', text): score += 5 
    if re.search(r'\b(dent|ding|scratch|paint chip|peeling)\b', text): score += 5 
    
    return score

def engineer_sharp_features(df):
    """
    Master function to generate all 'sharp' text features.
    Works on both Training Data (Many Rows) and Inference Data (Single Row).
    """
    df = df.copy()
    
    # 1. Ensure text columns exist (prevents crash on inference if JSON is incomplete)
    text_cols = ['Highlights', 'Equipment', 'Modifications', 'Known Flaws', 
                 'Recent Service History', 'Ownership History', 'Seller Notes', 'Other Items Included in Sale']
    
    for col in text_cols:
        if col not in df.columns:
            df[col] = ""
        df[col] = df[col].astype(str).fillna('').str.lower()

    # Create a blob for broad searching
    df['full_text_blob'] = df[text_cols].apply(lambda x: ' '.join(x), axis=1)

    # 2. Boolean Flags (The Multipliers)
    df['2_keys_ind'] = df['Other Items Included in Sale'].apply(
    lambda x: 1 if re.search(r'\b(2 keys|3 keys)\b', x) else 0
    )

    df['is_dry_climate_car'] = df['full_text_blob'].apply(
        lambda x: 1 if re.search(r'\b(california|arizona|texas|nevada|dry state|no rust)\b', x) else 0
    )

    df['is_project_car'] = df['full_text_blob'].apply(
        lambda x: 1 if re.search(r'\b(project|needs restoration|not running|tow away)\b', x) else 0
    )

    df['has_new_tires'] = df['Recent Service History'].apply(
    lambda x: 1 if re.search(r'\b(new tires|replaced tires|michelin|fresh rubber)\b', x) else 0
    )

    df['has_sport_seats'] = df['full_text_blob'].apply(
        lambda x: 1 if re.search(r'\b(recaro|sport seat|bucket seat|vader|wingback)\b', x) else 0
    )

    df['emissions_ind'] = df['Seller Notes'].apply(
        lambda x: 1 if re.search(r'\b(emissions)\b', x) else 0
    )

    df['loan_ind'] = df['Seller Notes'].apply(
        lambda x: 1 if re.search(r'\b(loan)\b', x) else 0
    )

    df['one_owner_ind'] = df['full_text_blob'].apply(
        lambda x: 1 if re.search(r'\b(one owner)\b', x) else 0
    )

    df['carfax_ind'] = df['Known Flaws'].apply(
        lambda x: 1 if re.search(r'\b(carfax)\b', x) else 0
    )

    # 3. Calculated Scores
    df['flaw_severity_score'] = df.apply(calculate_flaw_severity, axis=1)

    # 4. Recent Maintenance (Capex)
    df['recent_major_service'] = df['Recent Service History'].apply(
        lambda x: 1 if re.search(r'\b(timing belt|clutch|ims|head gasket|water pump|transmission replaced|engine replaced|)\b', x) else 0
    )

    # 5. Categorical extraction
    df['mod_status'] = df['Modifications'].apply(categorize_mods)

    # Drop the temporary blob to keep things clean
    df = df.drop(columns=['full_text_blob'])
    
    return df

def clean_date(x):
    """
    Robustly extracts date from strings like 'Feb 18, 2026 1:48 PM MST'.
    Ignores time and timezone.
    """
    if pd.isna(x):
        return None

    # NEW: If it's already a datetime or pandas Timestamp (from our sorting step), just return it
    if isinstance(x, datetime) or type(x).__name__ == 'Timestamp':
        return x

    x = str(x).strip()

    # REGEX STRATEGY: Look for "Mmm DD, YYYY" pattern
    # This matches "Feb 18, 2026" inside the longer string
    match = re.search(r'([A-Za-z]{3}\s+\d{1,2},\s+\d{4})', x)

    if match:
        date_str = match.group(1) # "Feb 18, 2026"
        try:
            return datetime.strptime(date_str, "%b %d, %Y")
        except ValueError:
            return None

    # Fallback for ISO dates (2026-02-18) just in case
    try:
        # Added .split(' ')[0] to safely strip off time if it exists in the string
        return datetime.strptime(x.split(' ')[0], "%Y-%m-%d")
    except ValueError:
        return None

def engineer_date_features(df, is_inference=False):
    """
    Extracts Year, Month, and calculates Car Age.
    NOW INCLUDES: Logic to grab Model Year from the "THIS... is a..." intro.
    """
    df = df.copy()
    
    # 1. Date Parsing (Auction End Date)
    if 'Auction_Date' not in df.columns:
        df['Auction_Date'] = None

    cleaned_dates = df['Auction_Date'].apply(clean_date)
    
    if is_inference:
        cleaned_dates = cleaned_dates.fillna(datetime.now())
    
    df['date_obj'] = pd.to_datetime(cleaned_dates)
    df['auction_year'] = df['date_obj'].dt.year
    df['auction_month'] = df['date_obj'].dt.month
    
    # 2. Extract Model Year (The "THIS... is a 1998" Logic)
    def extract_year_from_text(row):
        # Strategy A: Look for "THIS… is a 1998" in Highlights
        # We check for both ellipsis character (…) and standard dots (...)
        text = str(row.get('Highlights', ''))
        match = re.search(r'THIS[…\.]+\s+is\s+a\s+(\d{4})', text, re.IGNORECASE)
        if match:
            return float(match.group(1))

        return None

    df['model_year'] = df.apply(extract_year_from_text, axis=1)

    # 3. Calculate "Age at Sale"
    # If we still can't find the year, we default to the auction year (age=0) to prevent errors
    df['model_year'] = df['model_year'].fillna(df['auction_year'])
    
    df['car_age'] = df['auction_year'] - df['model_year']
    
    # Sanity Check: If age is negative (e.g. 2024 model sold in 2023), clamp to 0
    df['car_age'] = df['car_age'].apply(lambda x: max(x, 0))
        
    # Cleanup
    df = df.drop(columns=['date_obj', 'model_year']) # We drop model_year because 'car_age' is the better predictor
    
    return df

In [2]:
# --- Main Execution ---

# 1. Load Data
df = pd.read_csv("../data/cars_and_bids_full_history_v3.csv") # Replace with your new file name

# 2. Clean Target Variable (Price)
df['Sold_Price'] = df['Sold_Price'].apply(clean_currency)
df = df.dropna(subset=['Sold_Price'])

# --- TEMPORAL SORTING ---
# Sort chronologically before dropping the date so TimeSeries validation works later
# Remove the timezone abbreviation (e.g., " MST", " PST") using regex to avoid pandas warnings
df['Auction_Date'] = df['Auction_Date'].astype(str).str.replace(r'\s+[A-Z]{3,4}$', '', regex=True)
df['Auction_Date'] = pd.to_datetime(df['Auction_Date'])
df = df.sort_values('Auction_Date').reset_index(drop=True)

# 3. Clean Numerical Features
df['Mileage'] = df['Mileage'].apply(clean_mileage)

# 4. Clean Text Features
df['Model'] = df['Model'].apply(clean_model)
df['Title Status'] = df['Title Status'].apply(clean_and_group_title)
df['Seller Type'] = df['Seller Type'].apply(clean_seller_type)

# 5. Apply Color Cleaning
df['Exterior Color'] = df['Exterior Color'].apply(get_main_color)
df['Interior Color'] = df['Interior Color'].apply(get_main_color)

# 6. Feature Engineering
df['Transmission_Type'] = df['Transmission'].apply(clean_transmission_type)
df['Gears'] = df['Transmission'].apply(extract_gears)

engine_data = df['Engine'].apply(extract_engine_info)
df['Engine_Displacement_L'] = [x[0] for x in engine_data]
df['Engine_Cylinders'] = [x[1] for x in engine_data]

# APPLY TEXT FEATURE ENGINEERING
df['flaw_count'] = df['Known Flaws'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) and str(x).strip() != '' else 0)
df = engineer_sharp_features(df)
df = engineer_date_features(df, is_inference=False)
df = df.dropna(subset=['auction_year'])

# 7. Drop Unused Columns
cols_to_drop = ['Bids', 'Views', 'URL', 'Watchers', 'Transmission', 'Engine', 'Auction_Date', 'VIN', 'Location', 'Highlights', 'Equipment', 'Modifications', 'Known Flaws', 'Recent Service History', 'Ownership History', 'Other Items Included in Sale', 'Seller Notes'] 
df_cleaned = df.drop(columns=cols_to_drop)
pd.set_option('display.max_columns', None)
df_cleaned.head()



TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [7]:
# 1. Define Columns
# Only the continuous numeric columns
cols_to_impute = ['Gears', 'Mileage', 'Engine_Displacement_L', 'car_age']
one_hot_cols = ['Title Status', 'Seller Type', 'Drivetrain', 'Transmission_Type', 'Body Style', 'Engine_Cylinders', 'mod_status', 'auction_month']
label_cols = ['Exterior Color', 'Interior Color']

# 2. Train/Test Split 
X = df_cleaned.drop(columns=['Sold_Price'])
y = df_cleaned['Sold_Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Recombine temporarily to make transformations easier
train_df = X_train.copy()
train_df['Sold_Price'] = y_train

test_df = X_test.copy()
test_df['Sold_Price'] = y_test

# 3. Impute Missing Values (Using ONLY Training Data Statistics)
imputation_values = {}
    
# Grouped Imputation for technical specs
for col in ['Gears', 'Engine_Displacement_L']:
    # Impute by Model median
    train_df[col] = train_df.groupby('Model')[col].transform(lambda x: x.fillna(x.median()))
    # Fallback to Make median
    train_df[col] = train_df.groupby('Make')[col].transform(lambda x: x.fillna(x.median()))
    
    # Global fallback for anything still missing
    median_val = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_val)
    imputation_values[col] = median_val
    
    # Apply global fallback to test set directly for simplicity in deployment
    test_df[col] = test_df[col].fillna(median_val)

# Global mean imputation for standard numericals
for col in ['Mileage', 'car_age']:
    mean_val = train_df[col].mean()
    imputation_values[col] = mean_val
    train_df[col] = train_df[col].fillna(mean_val)
    test_df[col] = test_df[col].fillna(mean_val)

# 4. Target Encode 'Model' (Using ONLY Training Data)
model_target_map = train_df.groupby('Model')['Sold_Price'].mean()
make_target_map = train_df.groupby('Make')['Sold_Price'].mean()

train_df['Model_Target_Encoded'] = train_df['Model'].map(model_target_map)
test_df['Model_Target_Encoded'] = test_df['Model'].map(model_target_map)

train_df['Make_Target_Encoded'] = train_df['Make'].map(make_target_map)
test_df['Make_Target_Encoded'] = test_df['Make'].map(make_target_map)

# Fill any unseen models in the 20% test set with the global training mean to prevent NaNs
global_mean_price = train_df['Sold_Price'].mean()
test_df['Model_Target_Encoded'] = test_df['Model_Target_Encoded'].fillna(global_mean_price)
test_df['Make_Target_Encoded'] = test_df['Make_Target_Encoded'].fillna(global_mean_price)

train_df = train_df.drop(columns=['Model', 'Make'])
test_df = test_df.drop(columns=['Model', 'Make'])

# 5. One-Hot Encoding (Combine temporarily to ensure identical dummy columns in train/test)
combined_df = pd.concat([train_df, test_df], keys=['train', 'test'])
combined_encoded = pd.get_dummies(combined_df, columns=one_hot_cols, drop_first=False)

train_df_encoded = combined_encoded.xs('train').copy()
test_df_encoded = combined_encoded.xs('test').copy()

# 6. Label Encoding
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    train_df_encoded[col] = le.fit_transform(train_df_encoded[col])
    
    # Safe transform for test set (maps any weird unseen labels in the test split to -1)
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    test_df_encoded[col] = test_df_encoded[col].apply(lambda x: le_dict.get(x, -1))
    label_encoders[col] = le

# 7. Final Inspection and Save (Saves as two separate files now)
print("Train Data Shape:", train_df_encoded.shape)
print("Test Data Shape:", test_df_encoded.shape)

train_df_encoded.to_csv('../data/train_data.csv', index=False)
test_df_encoded.to_csv('../data/test_data.csv', index=False)

train_df_encoded.head()

Train Data Shape: (24619, 81)
Test Data Shape: (6155, 81)


,Bids,Views,Make,Mileage,Exterior Color,Interior Color,Gears,Engine_Displacement_L,2_keys_ind,is_dry_climate_car,is_project_car,has_new_tires,has_sport_seats,emissions_ind,loan_ind,one_owner_ind,carfax_ind,flaw_severity_score,recent_major_service,auction_year,car_age,Sold_Price,Model_Target_Encoded,Title Status_Alternate Doc,Title Status_Buyback,Title Status_Clean,Title Status_Mileage Issue,Title Status_Other,Title Status_Rebuilt/Salvage,Title Status_Unknown,Seller Type_Dealer,Seller Type_Private Party,Drivetrain_4WD/AWD,Drivetrain_Front-wheel drive,Drivetrain_Rear-wheel drive,Transmission_Type_Automatic,Transmission_Type_Manual,Body Style_Convertible,Body Style_Coupe,Body Style_Hatchback,Body Style_SUV/Crossover,Body Style_Sedan,Body Style_Truck,Body Style_Van/Minivan,Body Style_Wagon,Engine_Cylinders_Electric,Engine_Cylinders_H12,Engine_Cylinders_H2,Engine_Cylinders_H4,Engine_Cylinders_H6,Engine_Cylinders_I2,Engine_Cylinders_I3,Engine_Cylinders_I4,Engine_Cylinders_I5,Engine_Cylinders_I6,Engine_Cylinders_I8,Engine_Cylinders_Other,Engine_Cylinders_Rotary,Engine_Cylinders_Unknown,Engine_Cylinders_V10,Engine_Cylinders_V12,Engine_Cylinders_V2,Engine_Cylinders_V6,Engine_Cylinders_V8,Engine_Cylinders_W12,Engine_Cylinders_W8,mod_status_heavy_mod,mod_status_light_mod,mod_status_unknown_mod,auction_month_1,auction_month_2,auction_month_3,auction_month_4,auction_month_5,auction_month_6,auction_month_7,auction_month_8,auction_month_9,auction_month_10,auction_month_11,auction_month_12
25137,37.0,13056.000000,91,24100.0,13,1,5.0,2.0,0,0,0,0,1,1,0,0,0,0,1,2025,29.0,24000.0,23017.561224,False,False,True,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
21238,52.0,15252.000000,8,121000.0,11,1,6.0,2.7,1,1,0,0,0,0,0,0,0,0,1,2024,23.0,13500.0,13541.000000,False,False,True,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False
16374,30.0,13251.430852,86,35000.0,11,5,5.0,5.4,0,0,0,0,0,0,0,0,0,5,1,2020,14.0,9300.0,29172.104575,False,False,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
20486,17.0,13907.000000,52,35800.0,1,1,4.0,2.4,1,1,0,0,0,0,0,0,1,5,1,2023,21.0,11357.0,10032.125000,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False
29847,30.0,10075.000000,16,96800.0,2,13,5.0,2.7,0,0,0,0,0,0,0,0,0,0,1,2021,34.0,8750.0,13745.686613,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False


In [8]:
artifacts = {
    "label_encoders": label_encoders,
    "model_encoder_map": model_target_map,
    "make_encoder_map": make_target_map,
    "imputation_values": imputation_values, # Use these means in app.py
    "global_mean_price": global_mean_price  # Fallback for target encoding
}

with open('../deployment/encoding_artifacts_002.pkl', 'wb') as f:
    pickle.dump(artifacts, f)